<a href="https://colab.research.google.com/github/suyesha07/Stock_Price_Prediction_using_news_headlines/blob/main/SP_using_news_headline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob
from xgboost import XGBClassifier


In [2]:
def analize_sentiment(tweet):
    
    analysis = TextBlob((str(tweet)))     #defining the function which will find the plority of a sentence
    return analysis.polarity 


In [3]:
news= pd.read_csv('Combined_News_DJIA.csv')

train_news = news[news['Date'] < '2014-07-15']   # SPLITTING THE DATASET INTO TRAINING AND TESTING
test_news = news[news['Date'] > '2014-07-14']

train_news_list = []
for row in range(0,len(train_news.index)): # CONVERT THE TRAINNG DATASET OF 27 COLUMNS INTO ONE ELEMENT IN THE LIST FOR EACH DAY
    train_news_list.append(' '.join(str(k) for k in train_news.iloc[row,2:27]))
    

In [4]:
vectorize= CountVectorizer(min_df=0.01, max_df=0.8) # DEFINING THE VECTOR FUNCTION, SPECIFYING THR MIN AND MAX WORD FREQUENCY FILTER
news_vector = vectorize.fit_transform(train_news_list) # TRANSFORMING THE TRAINING DATASET INTO WORD FREQUENCY TRANFORMATION
print( "THE TABLE OF FREQUENCY WORD DISTRIBUTION" , news_vector.shape)

lr=LogisticRegression()
model = lr.fit(news_vector, train_news["Label"])

test_news_list = []
for row in range(0,len(test_news.index)):
    test_news_list.append(' '.join(str(x) for x in test_news.iloc[row,2:27]))# CONVERT THE TESTING DATASET OF 27 COLUMNS INTO ONE ELEMENT IN THE LIST FOR EACH DAY


THE TABLE OF FREQUENCY WORD DISTRIBUTION (1492, 4733)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [5]:
test_vector = vectorize.transform(test_news_list) # TRANSFORMING THE TESTING DATASET INTO WORD FREQUENCY TRANFORMATION

predictions = model.predict(test_vector)

pd.crosstab(test_news["Label"], predictions, rownames=["Actual"], colnames=["Predicted"])

accuracy1=accuracy_score(test_news['Label'], predictions)
print("the baseline model accuracy", accuracy1)


the baseline model accuracy 0.4607645875251509


In [6]:
words = vectorize.get_feature_names()
coefficients = model.coef_.tolist()[0]
coeffdf = pd.DataFrame({'Word' : words,'Coefficient' : coefficients})  # WORD DISTRIBUTION OF THE MODEL

coeffdf = coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
print("Top ten words according to the baseline model",coeffdf.head(10))
print("Last ten words according to the baseline model",coeffdf.tail(10))

Top ten words according to the baseline model           Word  Coefficient
3728      self     0.628003
4647      wing     0.532282
2090  hospital     0.532009
2392     kills     0.524826
284      among     0.516879
4387      turn     0.515314
762     cartel     0.513590
2929  olympics     0.509167
1146  damascus     0.505753
3585      rise     0.503551
Last ten words according to the baseline model            Word  Coefficient
3770        sex    -0.540544
1163         de    -0.541079
990       congo    -0.548451
4206     terror    -0.555603
4047   students    -0.563997
3653  sanctions    -0.569230
2100      hours    -0.572524
506       begin    -0.602589
4301      total    -0.610891
3626        run    -0.665577


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# Bi-gram

In [7]:
nvectorize = TfidfVectorizer(min_df=0.05, max_df=0.85,ngram_range=(2,2)) # DEFINING THE TFID TRANSFORMATION FUNCTION
news_nvector = nvectorize.fit_transform(train_news_list)

print(" TFID TRANSFOMATION DATAFRAME SHAPE",news_nvector.shape)

nmodel = lr.fit(news_nvector, train_news["Label"])


 TFID TRANSFOMATION DATAFRAME SHAPE (1492, 284)


In [8]:
test_news_list = []
for row in range(0,len(test_news.index)):
    test_news_list.append(' '.join(str(x) for x in test_news.iloc[row,2:27])) # CONVERT THE TESTING DATASET OF 27 COLUMNS INTO ONE ELEMENT IN THE LIST FOR EACH DAY
ntest_vector = nvectorize.transform(test_news_list)
npredictions = nmodel.predict(ntest_vector)

pd.crosstab(test_news["Label"], npredictions, rownames=["Actual"], colnames=["Predicted"])

accuracy2=accuracy_score(test_news['Label'], npredictions)
print(" Logistics Regression with Bigram and TFID",accuracy2)

 Logistics Regression with Bigram and TFID 0.5311871227364185


In [9]:
nwords = nvectorize.get_feature_names()
ncoefficients = nmodel.coef_.tolist()[0]
ncoeffdf = pd.DataFrame({'Word' : nwords, 
                        'Coefficient' : ncoefficients})
ncoeffdf = ncoeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
ncoeffdf.head(10)
ncoeffdf.tail(10)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Word,Coefficient
242,to help,-0.870909
262,us and,-0.880771
4,accused of,-0.952199
153,people are,-0.974838
20,around the,-1.009906
175,south africa,-1.019293
6,after being,-1.088850
244,to kill,-1.197320
188,the country,-1.225612
260,up in,-1.273317


# Random Forest - Bigram

In [10]:
#random forest - bigram

nvectorize = TfidfVectorizer(min_df=0.01, max_df=0.95,ngram_range=(2,2))
news_nvector = nvectorize.fit_transform(train_news_list)

rfmodel = RandomForestClassifier(random_state=55)  #DEFINNG THE RANDOM FOREST MODEL
rfmodel = rfmodel.fit(news_nvector, train_news["Label"])
test_news_list = []
for row in range(0,len(test_news.index)):
    test_news_list.append(' '.join(str(x) for x in test_news.iloc[row,2:27]))
ntest_vector = nvectorize.transform(test_news_list)

rfpredictions = rfmodel.predict(ntest_vector)
accuracyrf = accuracy_score(test_news['Label'], rfpredictions)
print("Random forest with tfid and bigram", accuracyrf)

Random forest with tfid and bigram 0.5372233400402414


# Naive Bayes

In [11]:
#Naive Bayes

nvectorize = TfidfVectorizer(min_df=0.05, max_df=0.8,ngram_range=(2,2)) #DEFINING THE NAIVE BAYS MODEL
news_nvector = nvectorize.fit_transform(train_news_list)

nbmodel = MultinomialNB(alpha=0.5)
nbmodel = nbmodel.fit(news_nvector, train_news["Label"])

test_news_list = []
for row in range(0,len(test_news.index)):
    test_news_list.append(' '.join(str(x) for x in test_news.iloc[row,2:27])) # CONVERT THE TESTING DATASET OF 27 COLUMNS INTO ONE ELEMENT IN THE LIST FOR EACH DAY
ntest_vector = nvectorize.transform(test_news_list)


In [12]:
nbpredictions = nbmodel.predict(ntest_vector)
nbaccuracy=accuracy_score(test_news['Label'], nbpredictions)
print("Naive Bayes accuracy: ",nbaccuracy)

Naive Bayes accuracy:  0.5291750503018109


# Gradient Boosting Classifier

In [13]:
#Gradient Boosting Classifier

gbmodel = GradientBoostingClassifier(random_state=52)  # DEFINING THE GARDIANT BOOSTING MODEL
gbmodel = gbmodel.fit(news_nvector, train_news["Label"])
test_news_list = []
for row in range(0,len(test_news.index)):
    test_news_list.append(' '.join(str(x) for x in test_news.iloc[row,2:27]))
ntest_vector = nvectorize.transform(test_news_list)

gbpredictions = gbmodel.predict(ntest_vector.toarray())
gbaccuracy = accuracy_score(test_news['Label'], gbpredictions)

from sklearn.metrics import confusion_matrix
print(" CONFUSION MATRIX OF THE GRADIANT BOOSTING ", confusion_matrix(test_news['Label'], gbpredictions))


print("Gradient Boosting accuracy: ",gbaccuracy)


 CONFUSION MATRIX OF THE GRADIANT BOOSTING  [[ 92 148]
 [ 74 183]]
Gradient Boosting accuracy:  0.5533199195171026


# Trigram

In [14]:
#trigram

n3vectorize = TfidfVectorizer(min_df=0.0004, max_df=0.115,ngram_range=(3,3)) # DEFINING THE TFID , TRIGRAM MODEL
news_n3vector = n3vectorize.fit_transform(train_news_list)

print(news_n3vector.shape)

n3model = lr.fit(news_n3vector, train_news["Label"])

test_news_list = []
for row in range(0,len(test_news.index)):
    test_news_list.append(' '.join(str(x) for x in test_news.iloc[row,2:27])) # CONVERT THE TESTING DATASET OF 27 COLUMNS INTO ONE ELEMENT IN THE LIST FOR EACH DAY
n3test_vector = n3vectorize.transform(test_news_list)
n3predictions = n3model.predict(n3test_vector)

pd.crosstab(test_news["Label"], n3predictions, rownames=["Actual"], colnames=["Predicted"])

accuracy3=accuracy_score(test_news['Label'], n3predictions)
print("TRIGARAM ACCURACY", accuracy3)

n3words = n3vectorize.get_feature_names()
n3coefficients = n3model.coef_.tolist()[0]
n3coeffdf = pd.DataFrame({'Word' : n3words, 
                        'Coefficient' : n3coefficients})
n3coeffdf = n3coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
print("trigram top ten word distibution", n3coeffdf.head(10))
print("trigram last ten word distibution", n3coeffdf.tail(10))    # trigram model word distribution 

(1492, 569061)
TRIGARAM ACCURACY 0.5171026156941649


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


trigram top ten word distibution                       Word  Coefficient
509383           to the us     0.201353
481307        the right to     0.170757
322285   nobel peace prize     0.165954
223934  human rights watch     0.159561
491158         this is the     0.154585
240342        in west bank     0.151582
230935        in china the     0.139373
518984         turn out to     0.137981
239146     in the occupied     0.132398
321584         no fly zone     0.127243
trigram last ten word distibution                          Word  Coefficient
183898      freedom of speech    -0.141552
356524        osama bin laden    -0.142172
371338  phone hacking scandal    -0.147765
497344              to be the    -0.148226
207018      has been arrested    -0.152082
509742              to try to    -0.152849
334728        of human rights    -0.170649
416292             said to be    -0.191760
48303        around the world    -0.195609
238814         in the country    -0.223890


# Sentiment Analysis

In [15]:
### sentiment analysis

train_sentiment=train_news
test_sentiment = test_news
train_sentiment =train_sentiment.drop(['Date', 'Label'], axis=1)
for column in train_sentiment:
    train_sentiment[column]=train_sentiment[column].apply(analize_sentiment)  #converting the train headlines into polarity scores
train_sentiment = train_sentiment+10  # removing negative co:efficient from the datset for better performance

test_sentiment =test_sentiment.drop(['Date', 'Label'], axis=1)
for column in test_sentiment:
    test_sentiment[column]=test_sentiment[column].apply(analize_sentiment) # converting the test headlines into ploarity 
test_sentiment=test_sentiment+10 # removing negative co:efficient from the datset for better performance

XGB_model= XGBClassifier()  # training the polarity score datset with DIJA 
gradiant=XGB_model.fit(train_sentiment, train_news['Label'])
y_pred= gradiant.predict(test_sentiment)


from sklearn.metrics import confusion_matrix
print(confusion_matrix(test_news['Label'], y_pred))
from sklearn.metrics import accuracy_score
print("Sentiment Accuracy",accuracy_score(test_news['Label'], y_pred))
from sklearn.metrics import f1_score
print("f1_score__",f1_score(test_news['Label'], y_pred, average='weighted'))

[[ 75 165]
 [ 70 187]]
Sentiment Accuracy 0.5271629778672032
f1_score__ 0.5057056775644162
